# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "output_data/cities.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ugoofaaru,5.6667,73.0000,81.86,76,20,8.79,MV,1638830252
1,Ko Samui,9.5357,99.9357,79.59,71,13,13.58,TH,1638830254
2,Oddur,4.1213,43.8894,71.69,78,51,9.35,SO,1638830256
3,Ushuaia,-54.8000,-68.3000,56.86,44,75,17.27,AR,1638830258
4,East London,-33.0153,27.9116,65.57,85,72,8.46,ZA,1638830261
...,...,...,...,...,...,...,...,...,...
546,Sisimiut,66.9395,-53.6735,21.97,76,100,19.33,GL,1638831498
547,Suez,29.9737,32.5263,61.74,61,100,2.53,EG,1638831501
548,Tocopilla,-22.0920,-70.1979,68.22,73,96,10.29,CL,1638831503
549,Assiut,27.1810,31.1837,53.78,58,100,3.74,EG,1638831505


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [7]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [8]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,Morón,-34.6534,-58.6198,76.05,72,0,5.99,AR,1638830120
128,Vao,-22.6667,167.4833,78.10,78,0,5.99,NC,1638830395
228,Cap Malheureux,-19.9842,57.6142,78.48,69,0,5.99,MU,1638830770
252,Mossamedes,-15.1961,12.1522,72.07,65,0,4.59,AO,1638830825
366,Salalah,17.0151,54.0924,70.88,34,0,6.15,OM,1638831086
435,Linguère,15.3953,-15.1193,75.42,29,0,6.91,SN,1638831246
475,Bahía Blanca,-38.7196,-62.2724,78.04,46,0,7.96,AR,1638831337
484,Beloha,-25.1667,45.0500,77.40,57,0,2.10,MG,1638831358
507,Gwadar,25.1216,62.3254,72.21,46,0,4.34,PK,1638831410
541,Maceió,-9.6658,-35.7353,78.24,83,0,3.44,BR,1638831487


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Issues with finding a hotel at Morón
Issues with finding a hotel at Beloha


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
71,Morón,-34.6534,-58.6198,76.05,72,0,5.99,AR,1638830120,NA
128,Vao,-22.6667,167.4833,78.10,78,0,5.99,NC,1638830395,OURE TERA beach resort
228,Cap Malheureux,-19.9842,57.6142,78.48,69,0,5.99,MU,1638830770,Zilwa Attitude Hotel
252,Mossamedes,-15.1961,12.1522,72.07,65,0,4.59,AO,1638830825,Hotel Chik Chik Namibe
366,Salalah,17.0151,54.0924,70.88,34,0,6.15,OM,1638831086,Salalah Gardens Hotel
435,Linguère,15.3953,-15.1193,75.42,29,0,6.91,SN,1638831246,Hôtel Amadou Seck
475,Bahía Blanca,-38.7196,-62.2724,78.04,46,0,7.96,AR,1638831337,Hotel Land Plaza
484,Beloha,-25.1667,45.0500,77.40,57,0,2.10,MG,1638831358,NA
507,Gwadar,25.1216,62.3254,72.21,46,0,4.34,PK,1638831410,PC Hotel Gawadar
541,Maceió,-9.6658,-35.7353,78.24,83,0,3.44,BR,1638831487,Maceió Mar Hotel


In [11]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
128,Vao,-22.6667,167.4833,78.10,78,0,5.99,NC,1638830395,OURE TERA beach resort
228,Cap Malheureux,-19.9842,57.6142,78.48,69,0,5.99,MU,1638830770,Zilwa Attitude Hotel
252,Mossamedes,-15.1961,12.1522,72.07,65,0,4.59,AO,1638830825,Hotel Chik Chik Namibe
366,Salalah,17.0151,54.0924,70.88,34,0,6.15,OM,1638831086,Salalah Gardens Hotel
435,Linguère,15.3953,-15.1193,75.42,29,0,6.91,SN,1638831246,Hôtel Amadou Seck
475,Bahía Blanca,-38.7196,-62.2724,78.04,46,0,7.96,AR,1638831337,Hotel Land Plaza
507,Gwadar,25.1216,62.3254,72.21,46,0,4.34,PK,1638831410,PC Hotel Gawadar
541,Maceió,-9.6658,-35.7353,78.24,83,0,3.44,BR,1638831487,Maceió Mar Hotel


In [12]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

8

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))